In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image

In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
    

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available else 'cpu')

In [4]:
orig_data = datasets.MNIST('./data/', train=True, download=True, transform=transform)

In [5]:
dloader = DataLoader(dataset=orig_data, batch_size=10)

In [6]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.l1 = nn.Linear(in_features=128, out_features=512)
        self.l2 = nn.Linear(in_features=512, out_features=784)
        
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
    
    def forward(self, x):
        # Expecting x to be of 128 * 1
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.tanh(x)
        return x

In [7]:
def getZ(device):
    var = torch.rand((10, 128))
    var = var.view(10, 1, 128)
    var = var.to(device)
    return var

In [8]:
var = getZ(device)

In [9]:
gen = Generator()

In [10]:
gen = gen.to(device)

In [11]:
x = gen(var)

In [12]:
x.size()

torch.Size([10, 1, 784])

In [13]:
class Disc(nn.Module):
    def __init__(self):
        super(Disc, self).__init__()
        self.l1 = nn.Linear(in_features=784, out_features=256)
        self.l2 = nn.Linear(in_features=256, out_features=1)
        
        self.relu = nn.LeakyReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.l1(x)
        x = self.relu(x)
        x = self.l2(x)
        x = self.sigmoid(x)
        return x

In [14]:
disc = Disc()

In [15]:
disc = disc.to(device)

In [16]:
op = disc(x)

In [17]:
op.size()

torch.Size([10, 1, 1])

In [18]:
discLoss = nn.BCELoss()

In [19]:
genLoss = nn.BCELoss()

In [20]:
gen = gen.to(device)

In [21]:
disc = disc.to(device)

In [22]:
dOptim = torch.optim.Adam(disc.parameters(), lr=0.0002)
gOptim = torch.optim.Adam(gen.parameters(), lr=0.0002)

In [23]:
def getLabels(bsize, device, orig=True):
    op = None
    if orig:
        op = torch.ones(bsize, 1)
    else:
        op = torch.zeros(bsize, 1)
    op = op.to(device)
    return op

In [24]:
epochs = 200

In [25]:
posLabel = getLabels(10, device)
fakeLabel = getLabels(10, device, False)

In [26]:
for e in range(epochs):
    print(e)
    for batch, _ in tqdm(dloader):
        batch = batch.to(device)
        batch = batch.view(10, 1, -1)
        
        # Train Discriminator
        
        dOptim.zero_grad()
        
        z = getZ(device)
        z = gen(z)
        
        fakeD = disc(z)
        origD = disc(batch)
        
        dFakeLoss = discLoss(fakeD, fakeLabel)
        dOrigLoss = discLoss(origD, posLabel)
        
        dLoss = -(dFakeLoss + dOrigLoss)
        dLoss.backward()
        
        # Train Generator
        
        gOptim.zero_grad()
        z = getZ(device)
        fakeD_gen = gen(z)
        
        disc_op = disc(fakeD_gen)
        gLoss = genLoss(fakeD, posLabel)
        
        gOptim.step()

  0%|          | 0/6000 [00:00<?, ?it/s]/home/sri/anaconda3/envs/ptorch/lib/python3.6/site-packages/torch/nn/functional.py:1594: UserWarning: Using a target size (torch.Size([10, 1])) that is different to the input size (torch.Size([10, 1, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))
  1%|          | 37/6000 [00:00<00:16, 361.58it/s]

0


  1%|          | 37/6000 [00:00<00:16, 361.35it/s]

1


  1%|          | 37/6000 [00:00<00:16, 368.36it/s]

...

  1%|          | 38/6000 [00:00<00:16, 370.58it/s]

198


  1%|          | 38/6000 [00:00<00:15, 372.88it/s]

199


100%|██████████| 6000/6000 [00:16<00:00, 370.31it/s]


In [ ]:
z = getZ(device)
z = gen(z)
z = z.cpu().detach()
z = z.numpy()
z.shape

In [ ]:
i = z[9].reshape(28, 28)
img = Image.fromarray(i, '1')

In [ ]:
img